# Test Develop Mock Circuit

In [3]:
from neuro_dmt.models.bluebrain.circuit.mock.cell import Position, Cell

In [4]:
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

(1.0, 1.0, 1.0)
[1. 1. 1.]
[[-0.59516615 -0.82632174 -0.87976773]
 [-0.41997555 -0.46442312  0.75757897]
 [ 0.10018043  0.97256148  0.22325506]]


In [5]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")

In [6]:
example_cell.as_dict

{'region': 'brain',
 'layer': 1,
 'nucleus': 'not-defined',
 'mtype': 'L23_BTC',
 'etype': 'not-defined',
 'morph_class': 'not-defined',
 'synapse_class': 'INH',
 'x': 1.0,
 'y': 2.0,
 'z': 3.0}

In [7]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition
circuit_composition.cell_density.head()

error         mean
mtype  layer                    
L1_DAC 1        0.0  3094.333333
       2        0.0     0.000000
       3        0.0     0.000000
       4        0.0     0.000000
       5        0.0     0.000000

In [8]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder

In [9]:
cb = CircuitBuilder(composition=circuit_composition)

In [10]:
cb.get_cell_density(mtype="L1_DAC", layer=1)

,,error,mean
mtype,layer,,
L1_DAC,1,0.0,3094.333333


In [11]:
print(circuit_composition.cell_density.head())
print(cb.get_number_cells(1, "L1_DAC"))

              error         mean
mtype  layer                    
L1_DAC 1        0.0  3094.333333
       2        0.0     0.000000
       3        0.0     0.000000
       4        0.0     0.000000
       5        0.0     0.000000
23


In [12]:
circuit_cells = cb.get_cells()

In [13]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

number of cells 12237
         etype  layer  morph_class   mtype      nucleus region synapse_class  \
0  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
1  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
2  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
3  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
4  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   

            x            y           z  
0  184.073794  1827.783745  104.385750  
1  242.210584  1747.434276   43.142952  
2  231.468702  1756.177760   19.581772  
3   81.923657  1722.898112   63.999505  
4   50.175557  1721.252435   42.855869  


In [15]:
cb